<p style='font-size: 16pt; text-align: center'> SPACE MISSION AND SPACECRAFT SUBSYSTEM ENGINEERING<br>ISM Semester – 1 (Sep 2022 – Jan 23)</p>

<p style='text-align: center;'>Attila Nemet<br>
2022-12-23</p>
<br>


# Final Project

## Part-2 Astrodynamics

## 1. Mars Mission

Designing a crewed Mars mission, based on chemical or nuclear thermal propulsion.
* calculating delta-V for a Mars mission
* Estimating the mass required for such a mission

### 1.1 Hohmann Transfer - Simplified

We are going to do a first approximation considering the following simplifications:
- the Earth and Mars are on circular orbits with the Sun in the center
- the Earth and Mars are orbiting in the same elliptic plane

In [1]:
r1 = 1     # AU
r2 = 1.524 # AU
a = (r1+r2)/2  # AU
a    # AU

1.262

In [2]:
T = (a**3)**0.5  # period of the transfer ellipse
T # years

1.417715319801546

In [3]:
t = T/2 # trip to Mars half of T period
t , t*365   # years, days

(0.708857659900773, 258.73304586378214)

 <center><img src="./img/Earth_Mars_Hohmann_Transfer.png" style="width: 550px; height: 500px;"></center>

In [4]:
G = 6.674e-11        # m3/kg/s2
m_Sun = 1.9885e30    # kg
m_Earth = 5.97217e24 # kg
m_Mars = 6.4171e23 # kg

mu_Sun = G * m_Sun          # m3/s2
mu_Earth = G * m_Earth      # m3/s2
mu_Mars = G * m_Mars        # m3/s2

M = m_Sun
m1 = m_Earth
m2 = m_Mars

Mu = mu_Sun
mu1 = mu_Earth
mu2 = mu_Mars



In [5]:
r1 = 149598023e3 # [m] Earth semi-major axis
r2 = 227939366e3 # [m]  Mars semi-major axis

a = (r1 + r2) / 2   # [m] semi-major axis of Elliptic Transfer orbit


In [6]:
# vp,va from vis-viva

vp = (Mu*(2/r1 - 1/a))**0.5
vp = vp/1e3

va = (Mu*(2/r2 - 1/a))**0.5
va = va/1e3

vp,va      # km/s

(32.729370588336494, 21.48048939492745)

In [7]:
# v1,v2 from vis-viva

v1 = (Mu/r1)**0.5
v1 = v1/1e3
v2 = (Mu/r2)**0.5
v2 = v2/1e3

v1, v2          # km/s

(29.784682279101773, 24.129383772394636)

In [8]:
delta_v1 = vp - v1
delta_v2 = v2 - va

delta_v1, delta_v2    # km/s

(2.9446883092347207, 2.6488943774671867)

In [9]:
# Sphere of influence

s1 = r1*(m1/M)**0.4
soi1 = s1/1e6             # [thousand km]
s2 = r2*(m2/M)**0.4
soi2 = s2/1e6             # [thousand km]
soi1, soi2

(924.6310655723548, 577.2243198669827)

In [10]:
delta_t_soi1 = s1/1000 / delta_v1        # [s]
delta_t_soi2 = s2/1000 / delta_v2        # [s]
delta_t_soi1/3600/24, delta_t_soi2/3600/24     # days

(3.6342550790734443, 2.5221228495662222)

## Patched Conics

### Hyperbolic planetary departure

 <center><img src="./img/hyperbolic_planetary_departure.png" style="width: 500px; height: 540px;"></center>

In [11]:
# patched conics
# semi-major axis of the exit hyperbola
a1hyp = 1 / (2/s1 - (delta_v1*1000)**2/mu1)
a1hyp / 1e6                     # thousand km

-51.04114787289372

In [12]:
r1p = (6371 + 500) * 1000    # radius of the parking orbit - 500 km height LEO
vp1hyp = (mu1 * (2/r1p - 1/a1hyp))**0.5
vp1hyp / 1000    # km/s

11.127796559942388

In [13]:
delta_v1hyp = vp1hyp - (mu1/r1p)**0.5
delta_v1hyp / 1000    # km/s

3.5114059719600363

In [14]:
from math import acos, pi, asin, cos, sin

In [15]:
beta1 = acos(1/(1 + r1p*(delta_v1*1000)**2/mu1))
beta1 * 180 / pi   # degrees

29.546086291229397

### Hyperbolic planetary arrival


 <center><img src="./img/hyperbolic_planetary_arrival.png" style="width: 577px; height: 500px;"></center>

In [16]:
# patched conics
# semi-major axis of the arrival hyperbola
a2hyp = 1 / (2/s2 - (delta_v2*1000)**2/mu2)
a2hyp / 1e6                     # thousand km

-6.235610057426045

In [17]:
r2p = (3390 + 200)*1000     # radius of the parking orbit - 200 km height LMO - Low Mars Orbit
vp2hyp = (mu2 * (2/r2p - 1/a2hyp))**0.5
vp2hyp / 1000    # km/s

5.543257752062432

In [18]:
delta_v2hyp = vp2hyp - (mu2/r2p)**0.5
delta_v2hyp / 1000    # km/s

2.089313843473748

In [19]:
beta2 = acos(1/(1 + r2p*(delta_v2*1000)**2/mu2))
beta2 * 180 / pi   # degrees

50.97512411098353

In [20]:
# eccentricity
ecc = 1 + r2p*(delta_v2*1000)**2/mu2
ecc

1.5881643835441333

In [21]:
# angle delta

𝛿 = 2 * asin(1/ecc)

𝛿 / pi * 180  # degrees

78.04975177803296

In [22]:
# double check 𝛿 + 2*beta2 == pi
𝛿 + 2*beta2

3.141592653589793

In [23]:
# distance 𝛥 between Mars trajectory and elliptic transfer orbit:

𝛥 = r2p * (1 + 2*mu2 / r2p / (delta_v2*1000)**2 )**0.5
𝛥 / 1000  # km

7530.798297236852

### Total Delta-V

In [24]:
# LEO speed at 500 km height

v_LEO_500 = (mu1/r1p)**0.5
v_LEO_500/1000    # circular orbit speed - [km/s]

7.616390587982352

In [25]:
# LMO speed at 200 km height

v_LMO_200 = (mu2/r2p)**0.5
v_LMO_200/1000    # circular orbit speed - [km/s]

3.453943908588685

In [26]:
# delta-v Earth-Mars-Transfer
delta_v_LEO = 9.4  # km/s
delta_v_LMO = 3.6  # km/s
delta_v_EMT = delta_v_LEO + delta_v1hyp/1000 + delta_v2hyp/1000 + delta_v_LMO
delta_v_EMT     # km/s

18.600719815433784

In [27]:
from math import log
ln = log

# Starship delta-v
v_e = 380 * 9.81 / 1000   # km/s
m_dry = 100  # tone
m_payload = 10  # tone
m_prop = 1200 # tone
mi = m_dry + m_payload + m_prop
mf = m_dry + m_payload
delta_v_Starship = v_e * ln(mi/mf)
delta_v_Starship  # km/s

9.234886583491487

In [28]:
# delta-v from LEO to landing on Mars
(delta_v1hyp + delta_v2hyp)/1000 + delta_v_LMO   # km/s

9.200719815433784

In [29]:
# astronaut mass budget for Earth-Mars transfer
daily_consumption = 5 # kg/day (2 kg food + 3 liter water/beverage)
m_astronaut = 100 # mass of an astronaut
t = 260  # days - Earth-Mars trip time
astronaut_mass_budget = t * daily_consumption + m_astronaut
astronaut_mass_budget   # kg

1400

In [30]:
# NTP delta-v  -- Nuclear Thermal Propulsion
g = 9.81  # gravitational acceleration
Isp = 900 # specific impulse
v_e = g * Isp / 1000   # exhaust velocity km/s
m_dry = 100  # tone
m_payload = 60  # tone
m_prop = 1200 # tone
mi = m_dry + m_payload + m_prop
mf = m_dry + m_payload
delta_v_NTP = v_e * ln(mi/mf)
delta_v_NTP  # km/s

18.894644157508576

In [31]:
# volume of LH2 for NTP
LH2_density = 71 # kg/m3
volume_LH2 = m_prop*1000 / LH2_density  # m3
volume_LH2  # m3

16901.408450704224

In [32]:
# LH2 storage dimensions for NTP
diameter = 15
height = volume_LH2 / ((diameter/2)**2 * pi)
height # m


95.64240711625635

Martian Colonist: The Hohmann Transfer

https://www.youtube.com/watch?v=LzsMjEMDpD4


https://ai-solutions.com/_freeflyeruniversityguide/patched_conics_transfer.htm

https://www.sciencedirect.com/topics/engineering/capture-orbit

Subject Zero Science: What it takes to go to Mars

https://www.youtube.com/watch?v=6-3IHfmwLJs

## 2. Advanced - Trajectory design for an interstellar object: 1I/‘Oumuamua

1I/’Oumuamua is the first interstellar object which was discovered in our solar system in 2017. The object is currently flying out of the solar system. What DeltaV would you need for a mission to ‘Oumuamua (without any flybys)? Do you think such a mission is feasible?


In [33]:
p_au = 0.255916 # perihelion in AU
au = 1.496e+11  # 1 AU in meter
p = p_au * au   # perihelion in meter
a_au = -1.2723  # semimajor axis in AU
a = a_au * au   # semimajor axis in meter
e = 1.20113     # eccentricity
v_inf = 26.33   # km/s
i = 122.74      # orbit inclination - degrees
LAAN = 24.597   # Longitude of Ascending Node - degrees
omega = 241.811 # Argument of perihelion - degrees
p_au, a_au, ecc

(0.255916, -1.2723, 1.5881643835441333)

In [34]:
# double check rp ~ p_au
rp = a_au*(1-e)
rp

0.25589769900000003

 <center><img src="./img/A_2017 U1 - Oumuamua - Trajectory 2018-03-20 _ orbital elements 6.png" </center>

In [35]:
# physical constants

g = 9.81      # m/s2
G = 6.674e-11        # m3/kg/s2

m_Sun = 1.9885e30    # kg
m_Earth = 5.97217e24 # kg

mu_Sun = G * m_Sun          # m3/s2
mu_Earth = G * m_Earth      # m3/s2

M = m_Sun
m1 = m_Earth

Mu = mu_Sun
mu1 = mu_Earth

au = 149_597_870.7  # 1 AU in km  - https://cneos.jpl.nasa.gov/glossary/au.html


In [36]:
from datetime import datetime, timedelta

class HypTra:
    def __init__(self, date0, r0, v_inf=26.33):
        self.date0 = datetime.strptime(date0, "%Y-%m-%d")
        self.r0 = r0                # km
        self.v_inf = v_inf          # km/s
        self.date = self.date0
        self.r = r0

        
    def step1day(self):
        self.date += timedelta(days=1)
        self.v_esc = (2 * Mu / (self.r * 1000))**0.5 / 1000   # km/s
        self.v = (self.v_inf**2 + self.v_esc**2)**0.5     # km/s
        self.dr = self.v * 24 * 3600                      # km
        self.r += self.dr
        

### 2.1.a First Iteration

In [37]:
ou = HypTra('2023-01-01', 3_084_966_115 * 1.609, v_inf = 26.33)
sc = HypTra('2023-12-31', au, v_inf = 31.33)


while ou.date != sc.date:
    ou.step1day()

while sc.r < ou.r:
    ou.step1day()
    sc.step1day()

print(f'''
      Mission start: {sc.date0.date()}
      Mission duration: {(sc.date-sc.date0).days/365:.1f} years
      The date of the encounter: {sc.date.date()}
      The distance of the encounter: {sc.r/au:.2f} AU\n''')



      Mission start: 2023-12-31
      Mission duration: 34.1 years
      The date of the encounter: 2058-02-04
      The distance of the encounter: 230.52 AU



### 2.1.b Second Iteration - Excess Velocity 10 km/s

In [38]:
ou = HypTra('2023-01-01', 3_084_966_115 * 1.609, v_inf = 26.33)
sc = HypTra('2023-12-31', au, v_inf = 36.33)


while ou.date != sc.date:
    ou.step1day()

while sc.r < ou.r:
    ou.step1day()
    sc.step1day()

print(f'''
      Mission start: {sc.date0.date()}
      Mission duration: {(sc.date-sc.date0).days/365:.1f} years
      The date of the encounter: {sc.date.date()}
      The distance of the encounter: {sc.r/au:.2f} AU\n''')



      Mission start: 2023-12-31
      Mission duration: 17.4 years
      The date of the encounter: 2041-05-11
      The distance of the encounter: 136.91 AU



### SC Leaving the Solar System

In [39]:
Isp = 330
mi = 2000
mf = 400
delta_v = Isp*g * ln(mi/mf)
delta_v /= 1000  # km/s
delta_v

5.210233353922913

In [40]:
Isp = 330
mi1 = 13000
mf1 = 2700
delta_v1 = Isp*g * ln(mi/mf)
delta_v1 /= 1000  # km/s
delta_v1

5.210233353922913

### 2.3 Velocity when Leaving Earth SOI

In [41]:
sc = HypTra('2023-12-31', au, v_inf = 31.33)
sc.step1day()
va = sc.v
print(f'va={va:.2f} km/s')

va=52.50 km/s


In [42]:
sc = HypTra('2023-12-31', au, v_inf = 36.33)
sc.step1day()
vb = sc.v
print(f'vb={vb:.2f} km/s')

vb=55.62 km/s


### 2.4 Projection of the Velocity when leaving Earth SOI

 <center><img src="./img/A_2017 U1 - Oumuamua - Trajectory 2018-03-20 _ theta 8.png" </center>

In [43]:
# theta_inf of the assimptote

theta_inf = acos(-1/e)
theta_inf = theta_inf * 180 / pi  # degrees
theta_inf

146.36151542066645

In [44]:
alpha = 180-((180-theta_inf)+(360-omega))
alpha

28.172515420666457

In [45]:
alpha1 = alpha - 3.5   # alpha is only an approximation of alpha1

In [46]:
v = va
v_x = v * cos(alpha1*pi/180)
v_yz = v * sin(alpha1*pi/180)
v_y = v_yz * cos((180-i)*pi/180)
v_z = v_yz * sin((180-i)*pi/180)
v_xy = (v_x**2 + v_y**2)**0.5
v_x, v_y, v_z, v_xy

(47.703521798780415, 11.851400446504602, 18.43213548903738, 49.15365382705672)

 <center><img src="./img/A_2017 U1 - Oumuamua - Trajectory 2023-06-23 _ mission 4.png" </center>

In [47]:
v_E = 29.78 # km/s  - average orbiting speed
va_SOI = ((v_xy - v_E)**2 + v_z**2)**0.5
va_SOI  # km/s

26.741018703423784

In [48]:
v = vb
v_x = v * cos(alpha1*pi/180)
v_yz = v * sin(alpha1*pi/180)
v_y = v_yz * cos((180-i)*pi/180)
v_z = v_yz * sin((180-i)*pi/180)
v_xy = (v_x**2 + v_y**2)**0.5
v_x, v_y, v_z, v_xy

(50.546786403369886, 12.557777379144905, 19.530742812916706, 52.08335039541967)

In [49]:
v_E = 29.78 # km/s  - average orbiting speed
vb_SOI = ((v_xy - v_E)**2 + v_z**2)**0.5
vb_SOI  # km/s

29.64606809823462

In [50]:
r1 = (6370+500)*1000
va_p = (2*mu1/r1 + (va_SOI*1000)**2)**0.5
print('va_p:', va_p/1000)
delta_va_LEO = va_p - (mu1/r1)**0.5
delta_va_LEO / 1000  # km/s

va_p: 28.829113413695428


21.212168523424364

In [51]:
r1 = (6370+500)*1000
vb_p = (2*mu1/r1 + (vb_SOI*1000)**2)**0.5
print('vb_p:', vb_p/1000)
delta_vb_LEO = vb_p - (mu1/r1)**0.5
delta_vb_LEO / 1000  # km/s

vb_p: 31.542432572774363


23.9254876825033

In [52]:
# difference between the slow and quick missions delta-v
vb_SOI - va_SOI, (delta_vb_LEO - delta_va_LEO)/1000      # km/s

(2.9050493948108347, 2.7133191590789356)

### 2.6 Mission Feasibility

In [53]:
delta_va_LEO /= 1000   # km/s
delta_va_LEO

21.212168523424364

In [54]:
delta_v_budget = delta_v_Starship + delta_v1 + delta_v - delta_va_LEO   # km/s
delta_v_budget   # km/s

-1.5568152320870503

### 2.7 Flyby

 <center><img src="./img/A_2017 U1 - Oumuamua - Trajectory 2023-06-25 _ Jupyter 1.png" ></center>

Earth - Jupiter position on 2023-06-25

Hein, A. M., Perakis, N., Eubanks, T. M., Hibberd, A., Crowl, A., Hayward, K., ... & Osborne, R. (2019). Project Lyra: Sending a spacecraft to 1I/’Oumuamua (former A/2017 U1), the interstellar asteroid. Acta Astronautica, 161, 552-561.

https://spacein3d.com/asteroid-oumuamua-live-tracker/

https://ntrs.nasa.gov/citations/19810001583
